<a href="https://colab.research.google.com/github/MengOonLee/Web_scraping/blob/master/Mydin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash
pip install -qU scrapy

In [2]:
%%bash
scrapy startproject Mydin

New Scrapy project 'Mydin', using template directory '/usr/local/lib/python3.7/dist-packages/scrapy/templates/project', created in:
    /content/Mydin

You can start your first spider with:
    cd Mydin
    scrapy genspider example example.com


In [3]:
%%bash
cd Mydin
scrapy genspider example example.com

Created spider 'example' using template 'basic' in module:
  Mydin.spiders.example


In [5]:
%%bash
cd Mydin
scrapy crawl example

2022-06-04 09:50:44 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: Mydin)
2022-06-04 09:50:44 [scrapy.utils.log] INFO: Versions: lxml 4.2.6.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.4.0, Python 3.7.13 (default, Apr 24 2022, 01:04:09) - [GCC 7.5.0], pyOpenSSL 22.0.0 (OpenSSL 3.0.3 3 May 2022), cryptography 37.0.2, Platform Linux-5.4.188+-x86_64-with-Ubuntu-18.04-bionic
2022-06-04 09:50:44 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'Mydin',
 'NEWSPIDER_MODULE': 'Mydin.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['Mydin.spiders']}
2022-06-04 09:50:44 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2022-06-04 09:50:44 [scrapy.extensions.telnet] INFO: Telnet Password: 310aff3ba6e28819
2022-06-04 09:50:44 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.